# Fundamental 25. 컴퓨팅 파워

## 프로세스
파이썬에서는 os 라는 모듈에서 프로세스 관련 정보를 얻을 수 있다.

In [1]:
import os

# process ID
print(os.getpid())

# user ID
print(os.getuid())

# group ID
print(os.getgid())

# 현재 작업중인 디렉토리
print(os.getcwd())

29
0
0
/aiffel/aiffel/AIFFEL-Fundamental


## 프로파일링

In [2]:
import timeit
        
def f1():
    s = set(range(100))

    
def f2():
    l = list(range(100))

    
def f3():
    t = tuple(range(100))


def f4():
    s = str(range(100))

    
def f5():
    s = set()
    for i in range(100):
        s.add(i)

def f6():
    l = []
    for i in range(100):
        l.append(i)
    
def f7():
    s_comp = {i for i in range(100)}

    
def f8():
    l_comp = [i for i in range(100)]
    

if __name__ == "__main__":
    t1 = timeit.Timer("f1()", "from __main__ import f1")
    t2 = timeit.Timer("f2()", "from __main__ import f2")
    t3 = timeit.Timer("f3()", "from __main__ import f3")
    t4 = timeit.Timer("f4()", "from __main__ import f4")
    t5 = timeit.Timer("f5()", "from __main__ import f5")
    t6 = timeit.Timer("f6()", "from __main__ import f6")
    t7 = timeit.Timer("f7()", "from __main__ import f7")
    t8 = timeit.Timer("f8()", "from __main__ import f8")
    print("set               :", t1.timeit(), '[ms]')
    print("list              :", t2.timeit(), '[ms]')
    print("tuple             :", t3.timeit(), '[ms]')
    print("string            :", t4.timeit(), '[ms]')
    print("set_add           :", t5.timeit(), '[ms]')
    print("list_append       :", t6.timeit(), '[ms]')
    print("set_comprehension :", t5.timeit(), '[ms]')
    print("list_comprehension:", t6.timeit(), '[ms]')

set               : 1.973096914589405 [ms]
list              : 0.9451897097751498 [ms]
tuple             : 0.8603362422436476 [ms]
string            : 0.5417465148493648 [ms]
set_add           : 6.453046699985862 [ms]
list_append       : 5.596351670101285 [ms]
set_comprehension : 6.369101023301482 [ms]
list_comprehension: 5.71142060495913 [ms]


이런 식으로 함수의 성능을 측정할 수 있다.

----
## 멀티 스레드

### 스레드 생성
파이썬에서 멀티스레드의 구현은 `threading` 모듈을 이용한다.  

아래 코드는 음식 배달과 그릇 찾기 2가지 작업을 순차적으로 수행하는 코드이다.

In [3]:
class Delivery:
	def run(self):
		print("delivery")

class RetriveDish:
	def run(self):
		print("Retriving Dish")

work1 = Delivery()
work2 = RetriveDish()

def main():
	work1.run()
	work2.run()

if __name__ == '__main__':
    main()

delivery
Retriving Dish


### 멀티 스레드
- `threading` 모듈을 import하고
- 클래스에 `Thread`를 상속받음

In [4]:
from threading import *

class Delivery(Thread):
	def run(self):
		print("delivery")

class RetriveDish(Thread):
	def run(self):
		print("Retriving Dish")

work1 = Delivery()
work2 = RetriveDish()

def main():
	work1.run()
	work2.run()

if __name__ == '__main__':
    main()

delivery
Retriving Dish


### 스레드 생성 확인

In [5]:
from threading import *

class Delivery:
    def run(self):
        print("delivering")

work1 = Delivery()
print(work1.run)

class Delivery(Thread):
    def run(self):
        print("delivering")

work2 = Delivery()
print(work2.run)

<bound method Delivery.run of <__main__.Delivery object at 0x7f1d249b7050>>
<bound method Delivery.run of <Delivery(Thread-6, initial)>>


위 코드를 실행해 확인해 보면, work2 는 다른 스레드(Thread-6)에 할당되었다.

### 스레드 생성

`threading` 모듈의 `Thread` 클래스를 상속받아서 구현할 수도 있지만 그대로 인스턴스화하여 스레드를 생성할 수도 있다.  

인스턴스화 하려면 `Thread` 클래스에 인자로 target과 args 값을 넣어준다. `args`에 넣어 준 파라미터는 스레드 함수의 인자로 넘어간다.  

```python
t = Thread(target=함수이름, args=())
```

`Thread`로 실행할 함수를 정의한 후 `start()` 를 통해 스레드를 실행한다.

In [6]:
from threading import *
from time import sleep

Stopped = False

def worker(work, sleep_sec):    # 일꾼 스레드입니다.
    while not Stopped:          # 그만 하라고 할때까지
        print('do ', work)      # 시키는 일을 하고
        sleep(sleep_sec)        # 잠깐 쉽니다.
    print('retired..')          # 언젠가 이 굴레를 벗어나면, 은퇴할 때가 오겠지요?
        
t = Thread(target=worker, args=('Overwork', 3))    # 일꾼 스레드를 하나 생성합니다. 열심히 일하고 3초간 쉽니다.
t.start()

do  Overwork
do  Overwork


스레드를 멈추기위해서는 스레드 함수가 루프를 돌 때는 꼭 멈춰야 할지를 체크하는 `flag`(여기서는 `Stopped`)를 체크하도록 설계해야한다.

In [7]:
# 이 코드 블럭을 실행하기 전까지는 일꾼 스레드는 종료하지 않습니다. 
Stopped = True    # 일꾼 일 그만하라고 세팅해 줍시다. 
t.join()          # 일꾼 스레드가 종료할때까지 기다립니다. 
print('worker is gone.')

retired..
worker is gone.


----
## 멀티프로세스
### 프로세스 생성

`Process` 인스턴스를 만든 뒤, `target` 과 `args` 파라미터에 각각 함수 이름과 함수 인자를 전달한다.

In [8]:
import multiprocessing as mp

def delivery():
    print('delivering...')

p = mp.Process(target=delivery, args=())
p.start()

delivering...


### 프로세스 사용
`Process` 클래스는 `start()`, `join()`, `terminate()` 같은 프로세스 동작 관련 메소드가 있다.

In [9]:
p = mp.Process(target=delivery, args=())
p.start() # 프로세스 시작
p.join() # 실제 종료까지 기다림 (필요시에만 사용)
p.terminate() # 프로세스 종료

delivering...


----
## 스레드/프로세스 풀 사용

사실 멀티스레드/프로세스 작업을 할 때 가장 많은 연산이 필요한 작업은 바로 이런 __스레드나 프로세스를 생성하고 종료__ 하는 일이다.  
스레드 풀을 만들면 각각의 태스크들에 대해 자동으로 스레드들을 할당하고 종료한다.  

풀을 만드는 방법은 크게 두가지가 있다.
- Queue를 사용해서 직접 만드는 방법
- `concurrent.futures` 라이브러리의 `ThreadPoolExecutor` , `ProcessPoolExecutor` 클래스를 이용하는 방법

### concurrent.futures 모듈

Java 언어의 ThreadPoolExecutor 를 파이썬에서 구현한 형태라고 생각하시면 편하다.  

크게 4가지 기능이 있다.
- `Executor` 객체
- `ThreadPoolExecutor` 객체
- `ProcessPoolExecutor` 객체
- `Future` 객체

#### `ThreadPoolExecutor`

`Executor` 객체를 이용하면 스레드 생성, 시작, 조인 같은 작업을 할 때, with 컨텍스트 관리자와 같은 방법으로 가독성 높은 코드를 구현할 수 있다.

```python
with ThreadPoolExecutor() as executor:
    future = executor.submit(함수이름, 인자)
```

앞서 한 Delivery 클래스를 예시로 들면 다음과 같다.

In [10]:
from concurrent.futures import ThreadPoolExecutor

class Delivery:
    def run(self):
        print("delivering")
w = Delivery()

with ThreadPoolExecutor() as executor:
    future = executor.submit(w.run)

delivering


#### `multiprocessing.Pool`

`multiprocessing.Pool.map` 을 통해 여러 개의 프로세스에 특정 함수를 매핑해서 병렬처리하도록 구현하는 방법이 널리 사용된다.  

In [11]:
from multiprocessing import Pool
from os import getpid

def double(i):
    print("I'm processing ", getpid())    # pool 안에서 이 메소드가 실행될 때 pid를 확인해 봅시다.
    return i * 2

with Pool() as pool:
      result = pool.map(double, [1, 2, 3, 4, 5])
      print(result)

I'm processing I'm processing I'm processing I'm processing I'm processing      777978

80
76

[2, 4, 6, 8, 10]


`double(i)` 이라는 메소드가 pool을 통해 각각 다른 pid를 가진 프로세스들 위에서 multiprocess로 실행되었다는 것을 확인할 수 있다.

----
## concurrent.futures 모듈의 ProcessPoolExecutor 를 이용해 병렬 프로그래밍 연습

소수(prime) 판별 문제로 PRIMES 변수에 선언된 숫자들이 소수인지 아닌지를 판별하는 문제

In [14]:
import math
import concurrent

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]


소수판별 함수 `is_prime`

In [15]:
def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

소수 판별 함수 호출
- `main()` 함수를 따로 작성하여 소수 판별 함수(`is_prime`)를 호출
- 맵-리듀스(map-reduce)스타일로 코드를 작성하고 `map()` 함수를 `ProcessPoolExecutor()` 인스턴스에서 생성된 executor 에서 실행
- `concurrent.futures` 라이브러리의 프로세스 풀에서 동작하게 하기 위해 `with` 문을 써서 구현

In [16]:
def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

병렬처리와 단일처리의 비교를 위해 코드를 아래와 같이 수정

- 프로파일링을 위한 시간 계산 코드를 추가
- 단일처리로 수행했을 때의 코드를 추가, 단일처리 프로파일링을 위한 시간 계산 코드를 추가

In [17]:
import time

def main():
    print("병렬처리 시작")
    start = time.time()
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))
    end = time.time()
    print("병렬처리 수행 시각", end-start, 's')

    start = time.time()
    for number, prime in zip(PRIMES, map(is_prime, PRIMES)):
        print('%d is prime: %s' % (number, prime))
    end = time.time()
    print("단일처리 수행 시각", end-start, 's')

### 전체 코드

In [18]:
main()

병렬처리 시작
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
병렬처리 수행 시각 0.7540721893310547 s
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
단일처리 수행 시각 2.807243585586548 s
